### **Downloading required libraries to use**

In [ ]:
!pip install face_recognition
!pip install dlib
!pip install imutils
# !pip install -v --install-option="--no" --install-option="DLIB_USE_CUDA" dlib

### **Import Libraries**

In [ ]:
from imutils import paths
import face_recognition
import argparse
import pickle
import cv2
import os

In [ ]:
Google colab drive

import glob
from google.colab.patches import cv2_imshow
from google.colab import drive
drive.mount('/content/drive',force_remount=True)


### **Face Detection using HOG OpenCV**

In [ ]:

print("quantifying faces")

# specify the path to the dataset
imagePaths = list(paths.list_images('/dataset'))

Encodings = []
Names = []


for (i, imagePath) in enumerate(imagePaths):
	
	name = imagePath.split(os.path.sep)[-2]
	image = cv2.imread(imagePath)
	rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

	boxes = face_recognition.face_locations(rgb,
		model="hog")

	encodings = face_recognition.face_encodings(rgb, boxes)

	for encoding in encodings:
		Encodings.append(encoding)
		Names.append(name)
	print(name)


print("serializing encodings...")
data = {"encodings": Encodings, "names": Names}
f = open('encodings4.txt', "wb")
f.write(pickle.dumps(data))
f.close()

### **Loading encodings**

In [ ]:
print("[INFO] loading encodings...")
data = pickle.loads(open('encodings4.txt', "rb").read())

### **Face Recognition**

In [ ]:
image = cv2.imread('test6.jpg')
print(image.shape)
rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
print("recognizing faces")
boxes = face_recognition.face_locations(rgb,
	model='hog')
encodings = face_recognition.face_encodings(rgb, boxes)

names = []

In [ ]:
for encoding in encodings:	
	
	matches = face_recognition.compare_faces(data["encodings"],
		encoding)
	name = "Unknown"

	if True in matches:
		
		matchedIdxs = [i for (i, b) in enumerate(matches) if b]
		counts = {}

		for i in matchedIdxs:
			name = data["names"][i]
			counts[name] = counts.get(name, 0) + 1

		name = max(counts, key=counts.get)
	
	names.append(name)

In [ ]:

for ((top, right, bottom, left), name) in zip(boxes, names):
	
	cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 255), 2)
	y = top - 15 if top - 15 > 15 else top + 15
	cv2.putText(image, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX,
		0.75, (0, 255, 255), 2)



cv2.imshow("Image", image)

#uncomment this to use in colab to show the image
# cv2_imshow(image)
cv2.waitKey(0)